In [1]:
import base64
import requests
import hashlib
import hmac
import datetime
from urllib import parse
import urllib.parse
from privateconfig import p_api_key, p_secret_key

In [2]:
def post_huobi(url, data, api_key, secret_key):
    timestamp = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')
    param = {"AccessKeyId": api_key,
             "SignatureVersion": "2",
             "SignatureMethod": "HmacSHA256",
             "Timestamp": timestamp
            }

    host = urllib.parse.urlparse(url).hostname
    path = urllib.parse.urlparse(url).path
    
    method = "POST"
    
    keys = sorted(param.keys())
    qs0 = '&'.join(['%s=%s' % (key, parse.quote(param[key], safe='')) for key in keys])
    payload0 = '%s\n%s\n%s\n%s' % (method, host, path, qs0)
    dig = hmac.new(secret_key.encode('utf-8'), msg=payload0.encode('utf-8'), digestmod=hashlib.sha256).digest()
    s = base64.b64encode(dig).decode()
    param["Signature"] =  s
    response = requests.post(url + "?" + urllib.parse.urlencode(param) , headers = {'Content-Type': 'application/json'}, json = data)
    print(response.text)

In [3]:
data = {'type': 0,
        'trade_type': 0,
        'create_date': 30,
        'status': 0,
        'symbol': 'ETH'
       }

In [4]:
post_huobi('https://api.hbdm.com/api/v1/contract_hisorders', data, p_api_key, p_secret_key)

{"status":"ok","data":{"orders":[{"order_id":810225244551110656,"contract_code":"ETH210326","symbol":"ETH","lever_rate":20,"direction":"sell","offset":"close","volume":509.000000000000000000,"price":1850.704000000000000000,"create_date":1613214351558,"order_source":"risk","order_price_type":1,"order_type":3,"margin_frozen":0E-18,"profit":-0.106251919785004660,"contract_type":"quarter","trade_volume":509.000000000000000000,"trade_turnover":5090.000000000000000000,"fee":0E-18,"trade_avg_price":1850.704,"status":6,"order_id_str":"810225244551110656","fee_asset":"ETH","liquidation_type":"3","update_time":1613214351000,"is_tpsl":0,"real_profit":-0.13064442980137565},{"order_id":810166248821739520,"contract_code":"ETH210326","symbol":"ETH","lever_rate":20,"direction":"buy","offset":"open","volume":509.000000000000000000,"price":1943.000000000000000000,"create_date":1613200285879,"order_source":"web","order_price_type":1,"order_type":1,"margin_frozen":0E-18,"profit":0E-18,"contract_type":"qua

In [20]:
def get_huobi(url, api_key, secret_key):
    timestamp = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')
    param = {"AccessKeyId": api_key,
             "SignatureVersion": "2",
             "SignatureMethod": "HmacSHA256",
             "Timestamp": timestamp
            }

    host = urllib.parse.urlparse(url).hostname
    path = urllib.parse.urlparse(url).path
    
    method = "GET"
    
    keys = sorted(param.keys())
    qs0 = '&'.join(['%s=%s' % (key, parse.quote(param[key], safe='')) for key in keys])
    payload0 = '%s\n%s\n%s\n%s' % (method, host, path, qs0)
    dig = hmac.new(secret_key.encode('utf-8'), msg=payload0.encode('utf-8'), digestmod=hashlib.sha256).digest()
    s = base64.b64encode(dig).decode()
    param["Signature"] =  s
    print(urllib.parse.urlencode(param))
    response = requests.get(url + "?" + urllib.parse.urlencode(param) , headers = {'Content-Type': 'application/json'})
    print(response.text)

In [21]:
get_huobi('https://api.hbdm.com/api/v1/contract_open_interest', p_api_key, p_secret_key)

AccessKeyId=8e454fd1-a0241651-cdgs9k03f3-41c25&SignatureVersion=2&SignatureMethod=HmacSHA256&Timestamp=2021-02-14T14%3A39%3A53&symbol=ETH&Signature=B6v1tzztRmMSd1iQ%2FBiPB5Ml1a25Gl03h6HTd2PbmZ0%3D
{"status":"ok","data":[{"volume":1130369.000000000000000000,"amount":15158.515701375487964983,"symbol":"BCH","contract_type":"next_quarter","contract_code":"BCH210625","trade_amount":138070.6564951371445943418259704270905814102,"trade_volume":9968706,"trade_turnover":99687060.000000000000000000},{"volume":342007.000000000000000000,"amount":654183.244070390206579954,"symbol":"EOS","contract_type":"next_week","contract_code":"EOS210226","trade_amount":10554546.0578045196132533917095429281826016334,"trade_volume":5673756,"trade_turnover":56737560.000000000000000000},{"volume":107904.000000000000000000,"amount":38534.528014170467004024,"symbol":"DOT","contract_type":"this_week","contract_code":"DOT210219","trade_amount":86969.1481291308937779156732408279465425856,"trade_volume":249254,"trade_turn